
# Generate tree of life for phylums
* Date: July 25, 2019
* location: /pylon5/mc5fr5p/hbagheri/09_Hadoop/seqRepIDTaxListOUTPUT


### Reference of phylums:  "A Higher Level Classification of All Living Organisms", PLOS' 2015

* List of phyla from this paper, see Table 2

* https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0119248#sec003


## Steps on Bridges generate the tree



#### we used ETE toolkit to get lineage and generate tree and translate taxonomy id to taxa name.

**TODO: python file here**


#### information about this analysis all are in the ```/Notebook_Hamid/phyl``` folder


## Boa <sub>g</sub> query

### list of tax ids that we want to create tree with 95% representative sequences and show the frequencies

```
s: Sequence = input;
clstrOut : output collection [string] of string;

getTaxList := function(seq: Sequence):string {
  tax_names := "";
  tax_ids :="";
 
  foreach(i:int; def(s.annotation[i])){
          if (s.annotation[i].tax_name !=""){
            tax_names = tax_names + s.annotation[i].tax_name + "$";     
            tax_ids = tax_ids + s.annotation[i].tax_id + " ";   
 
          }
  }
 return tax_ids + " == $" + tax_names;
};

foreach(i:int; def(s.cluster[i]))
 if (s.cluster[i].similarity==95 && s.cluster[i].representative){
  clstrOut [s.seqid] <<  getTaxList(s);

}


```

* we have 744237 tax ids in the 95% similarity
```
[hbagheri@login018 tax95]$ wc -l uniqtax95
744237 uniqtax95
```




### "Proposed hierarchical classification from superkingdom to order", PLOS' 2015

* we get list of tax ids that we are interested in to calculate the number of tax ids in their subtree, i.e, they are basically list of phylums


```
Crenarchaeota
Euryarchaeota
Acidobacteria
Aquificae
Armatimonadetes
Bacteroidetes
Caldiserica
Chlamydiae
Chlorobi
Chrysiogenetes
Cyanobacteria
Deferribacteres
Deinococcus-Thermus 
Dictyoglomi
Elusimicrobia
Fibrobacteres
Fusobacteria
Gemmatimonadetes
Lentisphaerae
Nitrospira
Planctomycetes
Proteobacteria
Spirochaetae
Synergistetes
Thermodesulfobacteria
Thermotogae
Verrucomicrobia
Actinobacteria
Chloroflexi 
Firmicutes
Tenericutes
Euglenozoa
Loukozoa
Metamonada
Percolozoa
Amoebozoa
Choanozoa  
Microsporidia 
Sulcozoa
N.N.
Cryptista
Haptophyta
Heliozoa
Ciliophora
Miozoa
Bigyra
Ochrophyta 
Pseudofungi 
Cercozoa
Retaria
Ascomycota
Basidiomycota
Chytridiomycota
Glomeromycota
Zygomycota
Glaucophyta
Rhodophyta
Chlorophyta
Charophyta
Anthocerotophyta
Bryophyta
Marchantiophyta
Tracheophyta
Cnidaria
Ctenophora
Placozoa
Porifera
Chaetognatha
Orthonectida
Rhombozoa
Arthropoda
Kinorhyncha
Loricifera
Nematoda
Nematomorpha
Onychophora
Priapula 
Tardigrada
Acanthocephala
Annelida
Brachiopoda
Bryozoa
Cycliophora
Entoprocta
Gastrotricha
Gnathostomulida
Micrognathozoa
Mollusca
Nemertea
Phoronida
Platyhelminthes
Rotifera
Sipuncula
Chordata
Echinodermata
Hemichordata
Xenacoelomorpha

```

#### Get list of Taxids from tax name
* NCBI taxonomy name/id : https://www.ncbi.nlm.nih.gov/Taxonomy/TaxIdentifier/tax_identifier.cgi



* we get list of all subtree tax ids for those interested tax ids , i.e. phylums
  * we needed to submit a job called join.sub on XSEDE to join these 2 files. It takes time. (Run time 00:48:50)

    ```
    [hbagheri@login018 seqIDTaxlist]$ cat join.sub
    #!/bin/bash

    #BATCH -J freq
    #SBATCH -o freq.o%j
    #SBATCH -p RM
    #SBATCH -N 1
    #SBATCH -n 28
    #SBATCH -t 48:00:00
    #SBATCH --mail-user=hbagheri@iastate.edu
    #SBATCH --mail-type=begin
    #SBATCH --mail-type=end
    #SBATCH --error=freq.err
    #SBATCH --output=freq.out

    cd /pylon5/mc5fr5p/hbagheri/09_Hadoop/seqIDTaxlist/
    grep -Fwf interested_taxids outLineage_taxids  > interested_list

    ```

### post processing:

```
# get the 3rd column which is the primary taxonomic assignment for 95%
grep "^clstrOut\[95\]" part-r-00000 | awk '{print $3}' part-r-00000 > tmp3column
# then we need to sort that file and calculate the counts
# this gives the tax Frequencies
sort tmp3column | uniq -c > taxFreq
# this is sample output

# we need to switch the columns
awk '{print $2, $1}' taxFreq > taxFreq2
```


## Post-processing on the Boa<sub>g</sub> output

In [52]:

from ete3 import NCBITaxa, PhyloTree, Tree, faces, AttrFace, TreeStyle, TextFace, NodeStyle

ncbi = NCBITaxa()
taxlist =list()
taxDic = dict()

def circular_layout(node):
    
    
    # If node is a leaf, add the nodes name and a its scientific name

    #FIXME: this code changes the color of leaves    
    lineage= ncbi.get_lineage(node.name)
    font_size = 30

    if 2 in lineage: 
        nameFace = faces.AttrFace("sci_name",  fgcolor="#0000ff", fsize=font_size)
        nameFace2 = faces.AttrFace("rank",  fgcolor="#0000ff", fsize=font_size) #fsize=20,
    elif 2157 in lineage:
        nameFace = faces.AttrFace("sci_name",  fgcolor="#800000", fsize=font_size)
        nameFace2 = faces.AttrFace("rank",  fgcolor="#800000", fsize=font_size) #fsize=20,
    else:
        nameFace = faces.AttrFace("sci_name",  fgcolor="#008000", fsize=font_size)
        nameFace2 = faces.AttrFace("rank",  fgcolor="#008000", fsize=font_size) #fsize=20,

    nameFace = faces.AttrFace("sci_name", fsize=font_size)
    nameFace2 = faces.AttrFace("rank") #fsize=20,
    
    
    rank = 100
    # add rank, frequency, and percentage to the nodes
    if node.is_leaf():
        #faces.add_face_to_node(nameFace, node, column=0)
        #faces.add_face_to_node(nameFace2, node, column=1)

        node.add_face(nameFace, column=2)
        node.add_face(TextFace(",     "), column=3)
        node.add_face(TextFace(taxDic[int(node.name)], fsize=font_size), column=4)
        

        #this gives the percentages of protein sequences for each clade
        percentages = 100 * float(taxDic[int(node.name)])/float(87986590) 
        
        if percentages < 1: 
            node.add_face(TextFace(" "), column=5)
            node.add_face(TextFace( " " ), column=6)
        else:
            node.add_face(TextFace(",     "), column=5)
            node.add_face( TextFace("(" + str(round(percentages, 2)) + "%)" , bold= True, fsize=font_size), column=6)
        
        #F = TextFace(node.name, tight_text=True)
        #node.add_face(F, column=7, position="branch-right")


def layout(node):
    
    
    # If node is a leaf, add the nodes name and a its scientific name

    lineage= ncbi.get_lineage(node.name)
    font_size = 15

    if 2 in lineage: 
        nameFace = faces.AttrFace("sci_name",  fgcolor="#0000ff", fsize=font_size)
        nameFace2 = faces.AttrFace("rank",  fgcolor="#0000ff", fsize=font_size) #fsize=20,
    elif 2157 in lineage:
        nameFace = faces.AttrFace("sci_name",  fgcolor="#800000", fsize=font_size)
        nameFace2 = faces.AttrFace("rank",  fgcolor="#800000", fsize=font_size) #fsize=20,
    else:
        nameFace = faces.AttrFace("sci_name",  fgcolor="#008000", fsize=font_size)
        nameFace2 = faces.AttrFace("rank",  fgcolor="#008000", fsize=font_size) #fsize=20,

   
    
    
    rank = 100
    # add rank, frequency, and percentage to the nodes
    if node.is_leaf():
        #faces.add_face_to_node(nameFace, node, column=0)
        #faces.add_face_to_node(nameFace2, node, column=1)

        node.add_face(nameFace, column=2)
        node.add_face(TextFace(", "), column=3)
        node.add_face(TextFace(taxDic[int(node.name)]), column=4)
        

        #this gives the percentages of protein sequences for each clade
        percentages = 100 * float(taxDic[int(node.name)])/float(87986590) 
        
        if percentages < 1: 
            node.add_face(TextFace(" "), column=5)
            node.add_face(TextFace( " " ), column=6)
        else:
            node.add_face(TextFace(",  "), column=5)
            node.add_face(TextFace("(" + str(round(percentages, 2)) + "%)" ), column=6, fsize=font_size)
        
        #F = TextFace(node.name, tight_text=True)
        #node.add_face(F, column=7, position="branch-right")


# interested_taxids is the output of previous step that is done on Bridges
with open('interested_taxids', "r") as f:
    for line in f:
        taxid = line.split()[0]
        seq_count = line.split()[1]

        rank = ncbi.get_rank([int(taxid)])

        taxlist.append(int(taxid))
        taxDic [int(taxid)] = int(seq_count)
        
        # we are interested in phylums
#         if rank[int(taxid)]=='phylum':
#             taxlist.append(int(taxid))
#             taxDic [int(taxid)] = int(seq_count)
        


tree = ncbi.get_topology(taxlist, intermediate_nodes=False)
tmp_tree2 = tree.get_ascii(attributes=["sci_name", "rank"])

# print(taxlist)
# print(tmp_tree2)

# print (tree.get_ascii(attributes=["sci_name", "rank", "name"], show_internal=False))


# Set dashed blue lines in all leaves
nst1 = NodeStyle()
nst1["bgcolor"] = "LightSteelBlue"
nst2 = NodeStyle()
nst2["bgcolor"] = "Moccasin"
nst3 = NodeStyle()
nst3["bgcolor"] = "DarkSeaGreen"

# nst4 = NodeStyle()
# nst4["bgcolor"] = "khaki"


n1 = tree.get_common_ancestor("1003038", "33154") #Eukaryota
n1.set_style(nst1)
    
n2 = tree.get_common_ancestor("28890", "28889") # Archaea
n2.set_style(nst2)

n3 = tree.get_common_ancestor("142182", "67814") # Bacteria
n3.set_style(nst3)

# n4 = tree.get_common_ancestor("4761", "1913637")
# n4.set_style(nst4)


ts = TreeStyle()
ts.show_leaf_name = False
# ts.branch_vertical_margin = 10 # 10 pixels between adjacent branches
ts.layout_fn = circular_layout
#ts.mode = "c" # draw tree in circular mode
ts.scale = 100



tree.render("%%inline", w=500, h=2500, units="mm", tree_style=ts)
# tree.render("itolSep7.png", w=400, units="mm", tree_style=ts)

#uncomment following 
# tree.show(tree_style=ts)





{'faces': [[1148.9045508967233,
   0.9287829837483618,
   1311.4415730526866,
   35.29375338243775,
   11,
   'Ctenophora'],
  [1311.4415730526866,
   12.074178788728704,
   1331.8747986951505,
   24.148357577457407,
   11,
   ',     '],
  [1331.8747986951505,
   0.9287829837483618,
   1350.4504583701178,
   35.29375338243775,
   11,
   '1'],
  [1350.4504583701178,
   12.074178788728704,
   1354.1655903051112,
   24.148357577457407,
   11,
   ' '],
  [1354.1655903051112,
   12.074178788728704,
   1357.8807222401047,
   24.148357577457407,
   11,
   ' '],
  [1148.9045508967233,
   35.29375338243775,
   1369.9549010288335,
   69.65872378112714,
   12,
   'Acanthocephala'],
  [1369.9549010288335,
   46.43914918741809,
   1390.3881266712974,
   58.51332797614679,
   12,
   ',     '],
  [1390.3881266712974,
   35.29375338243775,
   1408.9637863462647,
   69.65872378112714,
   12,
   '0'],
  [1408.9637863462647,
   46.43914918741809,
   1412.6789182812581,
   58.51332797614679,
   12,
   ' '

In [3]:

# percentages = 100 * float(total_seq)/float(87986590) 

# print(percentages)            
# print(round(percentages, 2))

for leaf in tree:
    leaf.add_features(seq_percentages=round(float(100 * taxDic[int(leaf.name)])/float(87986590), 2))

for leaf in tree:
    leaf.add_features(seq_count= taxDic[int(leaf.name)])

print (tree.get_ascii(attributes=["sci_name","name", "rank",  "seq_percentages",  "seq_count"], show_internal=True))
    
with open ("TreeAscii", "w") as outFile:
    outFile.write(tree.get_ascii(attributes=["sci_name", "name", "rank",  "seq_percentages", "seq_count"], show_internal=True))


                                                                                                                                                                                                                                                                                                          /-Ctenophora, 516519, genus, 0.0, 1
                                                                                                                                                                                                                                                       /Arthropoda, 6656, phylumNeoptera, 33340, infraclass
                                                                                                                                                                                                                                                      |                                                   \-Acanthocephala, 2316800, genus, 0.0, 0
                          

In [4]:
superkingdoms = [2, 2157, 2759] # Bacteria, Archaea, Eukarya
total_seq = 0
print (len(taxlist))
for tax in taxlist:
    lineage= ncbi.get_lineage(tax)
    
    # What else is not in the 3 superkingdoms
    #for sk in superkingdoms:
    if 2 not in lineage and  2157 not in lineage and 2759 not in lineage :
            print(lineage)
            total_seq +=taxDic[tax]
            print(ncbi.get_taxid_translator([tax]))
            rank = ncbi.get_rank([int(tax)])
            print(rank)
            print(taxDic[tax])
            print()


print(total_seq)

128
[1, 131567]
{131567: 'cellular organisms'}
{131567: 'no rank'}
87256753

[1]
{1: 'root'}
{1: 'no rank'}
87986590

175243343


## Bacteria (tax id = 2)

In [5]:
total_seq = 0
for tax in taxlist:
    lineage= ncbi.get_lineage(tax)
    # Bacteria
    if 2 in lineage:
        #print(taxDic[tax])
        print(lineage)
        total_seq +=taxDic[tax]
        print(ncbi.get_taxid_translator([tax]))

        
print(total_seq)

#this gives the percentages of protein sequences for each clade
percentages = 100 * float(total_seq)/float(87986590) 

print(percentages)            
print(round(percentages, 2))

[1, 131567, 2, 1783257]
{1783257: 'PVC group'}
[1, 131567, 2, 1783270, 142182, 219685]
{219685: 'Gemmatimonadetes'}
[1, 131567, 2, 1783257, 204428]
{204428: 'Chlamydiae'}
[1, 131567, 2, 1783257, 204428, 204429]
{204429: 'Chlamydiia'}
[1, 131567, 2, 40117, 203693, 189778]
{189778: 'Nitrospirales'}
[1, 131567, 2, 40117, 203693, 189778, 189779]
{189779: 'Nitrospiraceae'}
[1, 131567, 2, 1783270, 68336, 1090]
{1090: 'Chlorobi'}
[1, 131567, 2, 57723]
{57723: 'Acidobacteria'}
[1, 131567, 2, 74152]
{74152: 'Elusimicrobia'}
[1, 131567, 2, 1783270, 65842, 204430]
{204430: 'Fibrobacteria'}
[1, 131567, 2, 57723, 204432]
{204432: 'Acidobacteriia'}
[1, 131567, 2, 200938]
{200938: 'Chrysiogenetes'}
[1, 131567, 2, 200783, 187857]
{187857: 'Aquificae'}
[1, 131567, 2, 40117, 203693, 189778, 189779, 1234]
{1234: 'Nitrospira'}
[1, 131567, 2, 203691]
{203691: 'Spirochaetes'}
[1, 131567, 2, 40117, 203693]
{203693: 'Nitrospira'}
[1, 131567, 2, 1783270]
{1783270: 'FCB group'}
[1, 131567, 2, 1783257, 203682, 2

## Fungi (tax id =4751)

In [6]:
total_seq = 0
for tax in taxlist:
    lineage= ncbi.get_lineage(tax)
    # Fungi
    if 4751 in lineage:
        #print(taxDic[tax])
        print(lineage)
        total_seq +=taxDic[tax]
        print(ncbi.get_taxid_translator([tax]))

        
print(total_seq)

#this gives the percentages of protein sequences for each clade
percentages = 100 * float(total_seq)/float(87986590) 

print(percentages)            
print(round(percentages, 2))

[1, 131567, 2759, 33154, 4751, 112252, 1913637, 451507]
{451507: 'Mucoromycotina'}
[1, 131567, 2759, 33154, 4751, 451864, 4890]
{4890: 'Ascomycota'}
[1, 131567, 2759, 33154, 4751, 112252, 1913637, 214504]
{214504: 'Glomeromycotina'}
[1, 131567, 2759, 33154, 4751, 112252, 1913638, 451827]
{451827: 'Zoopagomycotina'}
[1, 131567, 2759, 33154, 4751, 112252, 1913638, 451828]
{451828: 'Kickxellomycotina'}
[1, 131567, 2759, 33154, 4751, 112252, 4761]
{4761: 'Chytridiomycota'}
[1, 131567, 2759, 33154, 4751, 112252, 1913638, 1264859]
{1264859: 'Entomophthoromycotina'}
[1, 131567, 2759, 33154, 4751, 112252, 6029]
{6029: 'Microsporidia'}
[1, 131567, 2759, 33154, 4751, 451864, 5204]
{5204: 'Basidiomycota'}
6665503
7.575589643830952
7.58


In [7]:
total_seq = 0
for tax in taxlist:
    lineage= ncbi.get_lineage(tax)
    # Archaea
    if 2157 in lineage:
        #print(taxDic[tax])
        print(lineage)
        total_seq +=taxDic[tax]
        print(ncbi.get_taxid_translator([tax]))

        
print(total_seq)

#this gives the percentages of protein sequences for each clade
percentages = 100 * float(total_seq)/float(87986590) 

print(percentages)            
print(round(percentages, 2))

[1, 131567, 2157, 28890]
{28890: 'Euryarchaeota'}
[1, 131567, 2157, 1783275, 28889, 183924]
{183924: 'Thermoprotei'}
[1, 131567, 2157, 1783275, 28889]
{28889: 'Crenarchaeota'}
1940885
2.2058872835053616
2.21


## Eukaryota (Tax id = 2759)

In [8]:
total_seq = 0
for tax in taxlist:
    lineage= ncbi.get_lineage(tax)
    # Eukaryota
    if 2759 in lineage:
        #print(taxDic[tax])
        total_seq +=taxDic[tax]
        print(ncbi.get_taxid_translator([tax]))

        
print(total_seq)

#this gives the percentages of protein sequences for each clade
percentages = 100 * float(total_seq)/float(87986590) 

print(percentages)            
print(round(percentages, 2))

{195505: 'Micrognathozoa'}
{10214: 'Rhombozoa'}
{37847: 'Priapulus'}
{3041: 'Chlorophyta'}
{10219: 'Hemichordata'}
{451507: 'Mucoromycotina'}
{69815: 'Cycliophora'}
{6447: 'Mollusca'}
{10205: 'Bryozoa'}
{6157: 'Platyhelminthes'}
{10197: 'Ctenophora'}
{10190: 'Rotifera'}
{6656: 'Arthropoda'}
{136419: 'Cercozoa'}
{4890: 'Ascomycota'}
{10232: 'Acanthocephala'}
{33317: 'Protostomia'}
{33313: 'Gastrotricha'}
{33310: 'Nematomorpha'}
{33154: 'Opisthokonta'}
{120557: 'Phoroniformea'}
{214504: 'Glomeromycotina'}
{2763: 'Rhodophyta'}
{1003038: 'Ctenophora'}
{10229: 'Chaetognatha'}
{1206794: 'Ecdysozoa'}
{10226: 'Placozoa'}
{28009: 'Choanoflagellida'}
{43120: 'Entoprocta'}
{451827: 'Zoopagomycotina'}
{6073: 'Cnidaria'}
{6072: 'Eumetazoa'}
{5878: 'Ciliophora'}
{451828: 'Kickxellomycotina'}
{33209: 'Orthonectida'}
{33208: 'Metazoa'}
{1312402: 'Xenacoelomorpha'}
{7711: 'Chordata'}
{38254: 'Glaucocystophyceae'}
{6433: 'Sipuncula'}
{6040: 'Porifera'}
{2830: 'Haptophyceae'}
{33213: 'Bilateria'}
{310840

In [9]:


total_seq = 0
for tax in taxlist:
    lineage= ncbi.get_lineage(tax)
    if 2759 in lineage:
        #print(taxDic[tax])
        total_seq +=taxDic[tax]
        print(ncbi.get_taxid_translator([tax]))
        print(lineage)
        print()

        
print(total_seq)

#this gives the percentages of protein sequences for each clade
percentages = 100 * float(total_seq)/float(87986590) 

print(percentages)            
print(round(percentages, 2))


{195505: 'Micrognathozoa'}
[1, 131567, 2759, 33154, 33208, 6072, 33213, 33317, 1206795, 195505]

{10214: 'Rhombozoa'}
[1, 131567, 2759, 33154, 33208, 10213, 10214]

{37847: 'Priapulus'}
[1, 131567, 2759, 33154, 33208, 6072, 33213, 33317, 1206794, 1215728, 33467, 2082883, 2082909, 37891, 37847]

{3041: 'Chlorophyta'}
[1, 131567, 2759, 33090, 3041]

{10219: 'Hemichordata'}
[1, 131567, 2759, 33154, 33208, 6072, 33213, 33511, 10219]

{451507: 'Mucoromycotina'}
[1, 131567, 2759, 33154, 4751, 112252, 1913637, 451507]

{69815: 'Cycliophora'}
[1, 131567, 2759, 33154, 33208, 6072, 33213, 33317, 1206795, 69815]

{6447: 'Mollusca'}
[1, 131567, 2759, 33154, 33208, 6072, 33213, 33317, 1206795, 6447]

{10205: 'Bryozoa'}
[1, 131567, 2759, 33154, 33208, 6072, 33213, 33317, 1206795, 10205]

{6157: 'Platyhelminthes'}
[1, 131567, 2759, 33154, 33208, 6072, 33213, 6157]

{10197: 'Ctenophora'}
[1, 131567, 2759, 33154, 33208, 6072, 10197]

{10190: 'Rotifera'}
[1, 131567, 2759, 33154, 33208, 6072, 33213, 3331